In [1]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager-core).
Your token has been saved to D:\.cache\token
Login successful


Load Dataset

In [4]:
#Load Datasets
from datasets import load_dataset
dataset_custom = load_dataset('audiofolder', data_dir='E:/HuggingFace/KoreanSpeech/')                              
dataset_zeroth = load_dataset("Bingsu/zeroth-korean")
dataset_fleurs = load_dataset('google/fleurs', 'ko_kr')
dataset_common_voice = load_dataset('mozilla-foundation/common_voice_12_0', 'ko')

Resolving data files:   0%|          | 0/12160783 [00:00<?, ?it/s]

Computing checksums:  10%|#         | 249434/2439752 [00:05<00:43, 49882.71it/s]

Computing checksums:   2%|2         | 240734/9721031 [00:05<03:16, 48142.37it/s]

Extracting data files:   0%|          | 0/9721031 [00:00<?, ?it/s]

In [3]:
print(dataset_custom)
print(dataset_zeroth)
print(dataset_fleurs)
print(dataset_common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 2439751
    })
})
DatasetDict({
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 457
    })
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 22263
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 2307
    })
    validation: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 226
    })
    test: Dataset({
        features: ['id', 'num_samples', 'path', 'audio', 'transcription', 'raw_transcription', 'gender', 'lang_id', 'language', 'lang_group_id'],
        num_rows: 382
    })
})
DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'u

PreProcessing Datasets

In [7]:
dataset_fleurs = dataset_fleurs.rename_column('transcription', 'text')
dataset_common_voice = dataset_polyai.rename_column("sentence", 'text')

use_cols=['audio', 'text']
dataset_fleurs = dataset_fleurs.remove_columns([col for col in dataset_fleurs.column_names['train'] if col not in use_cols])
dataset_polyai = dataset_polyai.remove_columns([col for col in dataset_polyai.column_names['train'] if col not in use_cols])


In [9]:
from datasets import DatasetDict, IterableDatasetDict
from datasets import concatenate_datasets
from datasets import interleave_datasets
from datasets import Audio

dataset_polyai = dataset_polyai.cast_column("audio", Audio(sampling_rate=16000))
dataset_fleurs = dataset_fleurs.cast_column("audio", Audio(sampling_rate=16000))
dataset_custom = dataset_custom.cast_column("audio", Audio(sampling_rate=16000))

stopping_strategy="all_exhausted"
data_dict = IterableDatasetDict()

data_dict["train"] = interleave_datasets(
    [
        dataset_fleurs['train'],
        dataset_polyai['train'],
        dataset_zeroth['train'],
        dataset_custom['train'],
    ],
        stopping_strategy=stopping_strategy
)

data_dict["test"] = interleave_datasets(
    [
        dataset_fleurs['test'],
        dataset_zeroth['test'],
    ],
        stopping_strategy=stopping_strategy
)



In [10]:
data_dict = DatasetDict(data_dict)
data_dict.column_names

{'train': ['audio', 'text'], 'test': ['audio', 'text']}

In [11]:
#Load FreatureExtravtor
from transformers import WhisperFeatureExtractor
#Load WhisperTokenizer
from transformers import WhisperTokenizer

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

#Prepare Dataset
def prepare_dataset(batch):
    from transformers import WhisperFeatureExtractor
    from transformers import WhisperTokenizer 
    feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
    tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

#Verify Tokenizer
input_str = data_dict['train'][0]['text']
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

#Combine To Create A WhisperProcessor
from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Korean", task="transcribe")

#ReSampling
from datasets import Audio

dataset = data_dict.cast_column("audio", Audio(sampling_rate=16000))

dataset = dataset.map(prepare_dataset, 
                            remove_columns=dataset.column_names["train"], num_proc=24,
                            )

Input:                 호주의 전 보수 정부는 인도와 중국 같은 나라는 온실 가스 방출 목표에 제한을 받지 않는 반면 석탄 수출에 크게 의존하고 있는 자국 경제에는 악영향을 미칠 것이라며 교토 의정서 비준을 거부했다
Decoded w/ special:    <|startoftranscript|><|ko|><|transcribe|><|notimestamps|>호주의 전 보수 정부는 인도와 중국 같은 나라는 온실 가스 방출 목표에 제한을 받지 않는 반면 석탄 수출에 크게 의존하고 있는 자국 경제에는 악영향을 미칠 것이라며 교토 의정서 비준을 거부했다<|endoftext|>
Decoded w/out special: 호주의 전 보수 정부는 인도와 중국 같은 나라는 온실 가스 방출 목표에 제한을 받지 않는 반면 석탄 수출에 크게 의존하고 있는 자국 경제에는 악영향을 미칠 것이라며 교토 의정서 비준을 거부했다
Are equal:             True


Map (num_proc=24):   0%|          | 0/66789 [00:00<?, ? examples/s]

Map (num_proc=24):   0%|          | 0/914 [00:00<?, ? examples/s]

In [12]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.0.0+cu118
True


In [13]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

import evaluate
metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

#Load a Pre-Trained Checkpoint
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-ko",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=3000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=100,
    eval_steps=100,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)

#Train

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

trainer.train()

Cloning https://huggingface.co/trueorfalse441/whisper-small-ko into local empty directory.
c:\Users\skim1\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/3000 [00:00<?, ?it/s]

`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


{'loss': 2.2082, 'learning_rate': 2.1000000000000002e-06, 'epoch': 0.01}
{'loss': 1.3446, 'learning_rate': 4.600000000000001e-06, 'epoch': 0.01}
{'loss': 0.6593, 'learning_rate': 7.100000000000001e-06, 'epoch': 0.02}
{'loss': 0.5063, 'learning_rate': 9.600000000000001e-06, 'epoch': 0.02}


  0%|          | 0/115 [00:00<?, ?it/s]

...

  0%|          | 0/115 [00:00<?, ?it/s]

{'eval_loss': 0.22629792988300323, 'eval_wer': 17.509552708473812, 'eval_runtime': 150.0777, 'eval_samples_per_second': 6.09, 'eval_steps_per_second': 0.766, 'epoch': 0.72}


Several commits (30) will be pushed upstream.


{'train_runtime': 14090.7439, 'train_samples_per_second': 3.406, 'train_steps_per_second': 0.213, 'train_loss': 0.13040668475627898, 'epoch': 0.72}


TrainOutput(global_step=3000, training_loss=0.13040668475627898, metrics={'train_runtime': 14090.7439, 'train_samples_per_second': 3.406, 'train_steps_per_second': 0.213, 'train_loss': 0.13040668475627898, 'epoch': 0.72})

Push To hub

In [15]:
kwargs = {
    "dataset_tags": "ko-speech",
    "dataset": "Speech Text",  # a 'pretty' name for the training dataset
    "dataset_args": "config: ko, split: test",
    "language": "ko",
    "model_name": "Whisper Small KR",  # a 'pretty' name for your model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
    "tags": "hf-asr-leaderboard",
}
trainer.push_to_hub(kwargs)

Several commits (32) will be pushed upstream.
The progress bars may be unreliable.
batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/trueorfalse441/whisper-small-ko'

Error pushing update to the model card. Please read logs and retry.
$batch response: Authorization error.
error: failed to push some refs to 'https://huggingface.co/trueorfalse441/whisper-small-ko'

